# Text Cleaning the Beige Books

In [2]:
# Code Borrowed from UC Berkeley's Data 8 lab/homework modules for simplicity
# http://data8.org/
import numpy as np
from datascience import *

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
from matplotlib import patches

In [3]:
beige_books = Table.read_table('beige_books.csv')
print("Pre-cleaning length: " + str(beige_books.num_rows))
beige_books

Pre-cleaning length: 25273


ID,Date,District,Sector Heading,Sector Text
1,5-20-1970,Atlanta,Summary of Economic Activity,The mood of our directors varies from pessimism to optim ...
2,5-20-1970,Boston,Summary of Economic Activity,Discussions Monday (May 18) and last Friday with First D ...
3,5-20-1970,Chicago,Summary of Economic Activity,In published statements and private conversations a deep ...
4,5-20-1970,Cleveland,Summary of Economic Activity,Economic activity in the District has been disrupted dur ...
5,5-20-1970,Dallas,Summary of Economic Activity,Continued inflation is the major concern of Eleventh Dis ...
6,5-20-1970,Kansas City,Summary of Economic Activity,Consensus Based on Discussions with Members of the Board ...
7,5-20-1970,Minneapolis,Summary of Economic Activity,Although indications of softening in the Ninth Federal R ...
8,5-20-1970,New York,Summary of Economic Activity,"In the Second Federal Reserve District, the Boards of Di ..."
9,5-20-1970,Philadelphia,Summary of Economic Activity,The business slowdown in the regional economy is now in ...
10,5-20-1970,Richmond,"Production, Sales, and Inventories","District survey respondents, except for those in certain ..."


In [4]:
beige_books = beige_books.where('ID', lambda s: s.isdigit())
beige_books.show(3)

ID,Date,District,Sector Heading,Sector Text
1,5-20-1970,Atlanta,Summary of Economic Activity,The mood of our directors varies from pessimism to optim ...
2,5-20-1970,Boston,Summary of Economic Activity,Discussions Monday (May 18) and last Friday with First D ...
3,5-20-1970,Chicago,Summary of Economic Activity,In published statements and private conversations a deep ...


# Cleaning Empty Sector Texts

In [5]:
empty_rows = beige_books.where("Sector Text", "nan")
empty_rows.show(3)
empty_row_ids = empty_rows.apply(lambda i: int(i), "ID")
empty_row_ids

ID,Date,District,Sector Heading,Sector Text
21,5-20-1970,San Francisco,Summary for Selected Economic Aspects,nan
30,5-20-1970,San Francisco,Inventories,nan
31,5-20-1970,San Francisco,Both higher and lower inventories are reported.,nan


array([   21,    30,    31,    93,   122,   136,   217,   259,   585,
         629,   711,   789,   842,  1007,  1287,  1366,  1989,  2456,
        2714,  3172,  5795,  5796,  5797,  5798,  5799,  5800,  5801,
        5802,  5803,  5804,  5805,  5806,  5807,  5808,  5809,  5810,
        5811,  5812,  5813,  5814,  6337,  6338,  6339,  6340,  6341,
        6342,  6343,  6344,  6345,  6346,  6347,  6348,  6349,  6350,
        6351,  6352,  6353,  6354,  6355,  6356,  6420,  6421,  6422,
        6423,  6424,  6425,  6426,  6427,  6428,  6429,  6430,  6431,
        6432,  6433,  6434,  6435,  6436,  6437,  6438,  6439,  6440,
        6441,  6442,  6443,  6444,  6798,  6826,  6827,  6828,  6829,
        6830,  6831,  6832,  6833,  6834,  6835,  6836,  6837,  6838,
        6839,  6840,  8368,  8731,  9527, 10411, 13392, 13941, 14403,
       16885, 18276, 18279, 18301, 19409, 20704, 21018, 21475, 21560,
       21563, 21588, 21602, 21673, 21721, 21810, 21836, 21901, 21990,
       22080, 22170,

In [20]:
ids_of_interest = []
empty_row_id = []
for i in empty_row_ids:
    empty_row_id.append(i)
    if i not in ids_of_interest:
        ids_of_interest.append(i - 1)
        ids_of_interest.append(i)
        ids_of_interest.append(i + 1)
    
ids_not_of_interest = [20, 21, 22, 29, 30, 31, 92, 93, 94, 121, 122, 123, 135, 136, 137]
ids_not_of_interest.extend([216, 217, 218, 260, 586, 258, 259, 584, 585, 628, 629, 630])
ids_not_of_interest.extend([])

for ids in ids_not_of_interest:
    ids_of_interest.remove(ids)
    
ids_of_interest = np.array(sorted(list(set(ids_of_interest))))

In [8]:
rows_of_interests = beige_books.where("ID", lambda i: int(i) in ids_of_interest)
rows_of_interests.show(300)

ID,Date,District,Sector Heading,Sector Text
710,1-15-1975,Atlanta,Summary of Economic Activity,"While the Sixth District economy remains below par, seve ..."
711,1-15-1975,Atlanta,capital-spending plans.,nan
712,1-15-1975,Boston,Summary of Economic Activity,"In New England the mood is bad, but directors note that ..."
788,7-9-1975,Kansas City,Summary of Economic Activity,A survey of a number of Tenth District purchasing manage ...
789,7-9-1975,Kansas City,1-year CD's.,nan
790,7-9-1975,Minneapolis,Summary of Economic Activity,"At midyear, the economic decline in the Ninth District a ..."
841,11-12-1975,Philadelphia,Summary of Economic Activity,"Economic activity in the Third District is growing, but ..."
842,11-12-1975,Philadelphia,tax-exempt income.,nan
843,11-12-1975,Richmond,Summary of Economic Activity,The November survey of Fifth District business condition ...
1006,1-13-1977,Chicago,Summary of Economic Activity,The economic outlook in the Seventh District continues t ...


In [9]:
ids_to_drop = [21]
h_h = [(30, 31)]
h_sh = [(92, 93), (121, 122), (135, 136), (216, 217), (258, 259), (584, 585), (628, 629)]
h_sh.extend([])

In [11]:
def merge_rows(id_1, id_2, table):
    new_row = []
    row_1 = table.where("ID", id_1).row(0)
    row_2 = table.where("ID", id_2).row(0)
    new_row.append(row_1[0], row_1[1], row_1[2], row_1[3])
    new_row.append(row_1[0] + row_2[4])

# Cleaning Strange Sector Headings

In [12]:
strange_headings = beige_books.where("Sector Heading", lambda s: len(s) < 5 and s != "Coal" and s != "Fuel")
strange_headings.show(3)
strange_heading_ids = strange_headings.apply(lambda i: int(i), "ID")
strange_heading_ids

ID,Date,District,Sector Heading,Sector Text
1989,8-11-1981,Cleveland,<>,nan
2456,6-23-1982,Cleveland,<>,nan
2714,12-15-1982,Chicago,.,nan


array([ 1989,  2456,  2714,  5796,  5797,  5798,  5801,  5802,  5803,
        5805,  5806,  5807,  5809,  5810,  5811,  5813,  5814,  5815,
        6338,  6339,  6342,  6343,  6344,  6346,  6347,  6348,  6350,
        6351,  6352,  6354,  6355,  6356,  6421,  6422,  6426,  6427,
        6428,  6429,  6431,  6432,  6433,  6434,  6436,  6437,  6438,
        6439,  6441,  6442,  6443,  6444,  6502,  6798,  6827,  6830,
        6831,  6832,  6834,  6835,  6836,  6838,  6839,  6840,  8368,
        8731, 10411, 14134, 14403, 18276, 19409, 20704, 21426, 21475,
       21560, 21673, 22957, 24843, 25175, 25208, 25209, 25210])

In [13]:
strange_headings.show(300)
strange_heading_id = []
for i in strange_heading_ids:
    strange_heading_id.append(i)

ID,Date,District,Sector Heading,Sector Text
1989,8-11-1981,Cleveland,<>,nan
2456,6-23-1982,Cleveland,<>,nan
2714,12-15-1982,Chicago,.,nan
5796,6-15-1988,Richmond,:,nan
5797,6-15-1988,Richmond,Rise,nan
5798,6-15-1988,Richmond,Slow,nan
5801,6-15-1988,Richmond,40%,nan
5802,6-15-1988,Richmond,10%,nan
5803,6-15-1988,Richmond,50%,nan
5805,6-15-1988,Richmond,23%,nan


In [14]:
id_removal = strange_heading_ids[:50]
id_removal

array([1989, 2456, 2714, 5796, 5797, 5798, 5801, 5802, 5803, 5805, 5806,
       5807, 5809, 5810, 5811, 5813, 5814, 5815, 6338, 6339, 6342, 6343,
       6344, 6346, 6347, 6348, 6350, 6351, 6352, 6354, 6355, 6356, 6421,
       6422, 6426, 6427, 6428, 6429, 6431, 6432, 6433, 6434, 6436, 6437,
       6438, 6439, 6441, 6442, 6443, 6444])

In [16]:
no_strange_headings_beige_books = False

# Cleaning Strange Dates

In [17]:
def isNotDate(s):
    s = s.split("-")
    if len(s) != 3:
        return True
    if not s[0].isdigit():
        return True
    if not s[1].isdigit():
        return False
    if not s[2].isdigit():
        return True
    return False

strange_dates = beige_books.where("Date", isNotDate)
strange_dates.show(300)

strange_date_id = []
for i in strange_dates:
    strange_date_id.append(i)

ID,Date,District,Sector Heading,Sector Text
132,"1-1-,",Boston,Summary of Economic Activity,"The January 12, 1971 Boston report is not available."
395,"11-1-,",Boston,Summary of Economic Activity,Reports from our directors indicate business is continui ...
397,"11-1-,",Cleveland,Summary of Economic Activity,The thrust of recent statistical evidence and informatio ...
398,"11-1-,",Dallas,Summary of Economic Activity,Most major indicators of economic activity in the Eleven ...
15959,"1-1-,",Cleveland,Summary of Economic Activity,"For the final six weeks of 2004, economic conditions wer ..."
15960,"1-1-,",Cleveland,Manufacturing,Production at the District's durable goods manufacturers ...
15961,"1-1-,",Cleveland,Retail Sales,The economic environment for retailers in the District w ...
15962,"1-1-,",Cleveland,Construction,Residential builders reported continuing declines in act ...
15963,"1-1-,",Cleveland,Banking,"District banks described loan demand as steady, in gener ..."
15964,"1-1-,",Cleveland,Trucking and Shipping,Demand for trucking and shipping services remained stron ...


# Final Table

In [34]:
bad_flags = empty_row_id + strange_date_id + strange_heading_id
final = beige_books.where("ID", lambda s: int(s) not in bad_flags)

In [35]:
print("Post-cleaning length: " + str(final.num_rows))

Post-cleaning length: 25122


In [38]:
len(list(set(bad_flags)))

152

In [39]:
final.to_csv("cleaned.csv")